In [1]:
!pip install langchain

In [2]:
!pip install youtube-transcript-api

In [3]:
!pip install chromadb

In [4]:
!pip install langchain-openai

In [5]:
!pip install langchain-community

In [6]:
!pip install gradio

In [7]:
!pip install openai

In [8]:
!pip install whisper

In [9]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_video_transcript(video_id):
    """
    Fetch transcript for a given YouTube video ID.
    """
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = " ".join([item['text'] for item in transcript if 'text' in item])
        return transcript_text
    except Exception as e:
        print(f"Error retrieving transcript for video {video_id}: {e}")
        return None

# Example usage with a list of video IDs
video_ids = ['2u4ItZerRac', 'I2zF1I60hPg', '8xqSF-uHCUs', 'LtmS-c1pChY', 'sJNxT-I7L6s']
transcripts = {}

# Fetch and print the transcript for each video
for video_id in video_ids:
    transcript = get_video_transcript(video_id)
    if transcript:
        transcripts[video_id] = transcript
    else:
        print(f"No transcript available for video {video_id}")

# Display the fetched transcripts
for video_id, transcript in transcripts.items():
    print(f"Transcript for {video_id}:")
    print(transcript[:500])  # Display the first 500 characters of each transcript for preview

Transcript for 2u4ItZerRac:
[Music] the pros and cons of living in Germany today we're going to talk about the pros and cons of living in Germany Germany is an amazing country with a strong economy lots of culture great infrastructure and friendly people but that doesn't mean it's the right place for everyone in this video we'll look at all aspects of life in Germany so you can decide if it's the right move for you so let's get started Pros living in Germany is a great experience it offers many advantages both long and sho
Transcript for I2zF1I60hPg:
should you move to Germany that's an easy question to a hard answer but I'm going to try my [Music] best hi my name is Fernando I have been living in Munich Germany for eight years now and yeah I have a lot of experience I say and I'm very happy to kind of like give you some ideas and some stories to will probably help you make a better decision if you should move to Germany or not the first thing of course is that there are many reasons w

In [10]:
import chromadb
from sentence_transformers import SentenceTransformer

# Initialize ChromaDB
chroma_client = chromadb.Client()

# Create a collection in ChromaDB
collection = chroma_client.create_collection("youtube_video_transcripts3")

# Initialize the SentenceTransformer model for embedding text
sentence_model = SentenceTransformer('all-mpnet-base-v2')

def embed_text(text):
    """
    Convert text into embeddings using SentenceTransformer.
    """
    return sentence_model.encode([text])[0]

def upsert_to_chromadb(video_id, transcript_text):
    """
    Store video transcript embeddings in ChromaDB.
    """
    embedding = embed_text(transcript_text)
    collection.upsert(
        ids=[video_id],  # Adding the 'ids' argument with the video_id
        embeddings=[embedding],
        metadatas=[{"video_id": video_id, "transcript": transcript_text}]
    )

# Upsert all transcripts into ChromaDB
for video_id, transcript in transcripts.items():
    upsert_to_chromadb(video_id, transcript)
    print(f"Stored embedding for {video_id}")

c:\Users\mehak\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Stored embedding for 2u4ItZerRac
Stored embedding for I2zF1I60hPg
Stored embedding for 8xqSF-uHCUs
Stored embedding for LtmS-c1pChY
Stored embedding for sJNxT-I7L6s


In [11]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [12]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key from the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = openai_api_key


# Now you can use the OpenAI API key in your script
from youtube_transcript_api import YouTubeTranscriptApi
import chromadb
from sentence_transformers import SentenceTransformer
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import openai

# Define the LangChain Prompt Template for QA
QA_TEMPLATE = """
Use the following context from a YouTube video transcript to answer the question:

Context: {context}

Question: {question}

Answer:
"""
qa_prompt = PromptTemplate(input_variables=["context", "question"], template=QA_TEMPLATE)

# Initialize the LLM (using GPT-4 mini model)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Combine the provided transcripts into a single context
combined_context = """

[Music] the pros and cons of living in Germany today we're going to talk about the pros and cons of living in Germany Germany is an amazing country with a strong economy lots of culture great infrastructure and friendly people but that doesn't mean it's the right place for everyone in this video we'll look at all aspects of life in Germany so you can decide if it's the right move for you so let's get started Pros living in Germany is a great experience it offers many advantages both long and sho
should you move to Germany that's an easy question to a hard answer but I'm going to try my [Music] best hi my name is Fernando I have been living in Munich Germany for eight years now and yeah I have a lot of experience I say and I'm very happy to kind of like give you some ideas and some stories to will probably help you make a better decision if you should move to Germany or not the first thing of course is that there are many reasons why you are probably moving to Germany either you got a a 
here in Germany the government is meeting this week to discuss new measures to more easily Deport migrants whose Asylum application was rejected or who have been convicted of certain crimes it's a controversial Topic in some areas of Germany but not in parts of the East where the anti-immigration alternative for Germany is the strongest political party many newly arrived migrants there worry that they are no longer welcome here because of war and I want to have a safe life here I'm 100% confiden
the alternative for Germany afd party has unveiled a radical new approach to immigration policy signaling a potential shift in the nation's migration landscape Party leaders have proposed stringent measures that could dramatically reshape Germany's approach to foreign Nationals and border control afd leadership has outlined a comprehensive strategy targeting what they describe as excessive immigration the proposed plan includes significant restrictions on Asylum Seekers family reunification and
"[Music] the pros and cons of living in Germany today we're going to talk about the pros and cons of living in Germany Germany is an amazing country with a strong economy lots of culture great infrastructure and friendly people but that doesn't mean it's the right place for everyone in this video we'll look at all aspects of life in Germany so you can decide if it's the right move for you so let's get started Pros living in Germany is a great experience it offers many advantages both long and shortterm that make it an attractive destination for those considering making the move here are some of the pros of living in Germany one free education for more than 25 years Germany has offered tuition-free higher education at public universities with no charge for most courses and little no cost for administrative fees this means that students coming from all over the world can pursue their degree programs without worrying about expensive tuition costs moreover those studying in certain Fields even have access to Grants and scholarships which cover living expenses as well in addition to this incredible benefit German universities also provide high quality instruction from some of the finest academic Minds in Europe with famous institutions such as humbold University Berlin leading the way in research and Innovation you are sure to receive an excellent standard of learning whether you're taking up a course on engineering or literature two drinkable tap water living in Germany certainly comes with a lot of perks from the delicious sausages to the breathtaking cities but one perk that doesn't often get mentioned is how drinkable tap water in Germany is it's true and it's really quite remarkable of course not all German towns have perfect tap water but by far most do in fact many Germans believe their drinking water is so good because it's monitored more stringently than other countries and for sure there are no worries about added chemicals or nasty tastes what could be better three unemployment benefits if find themselves out of work due to whatever reason don't fret your well-being is taken care of thanks to the German government's generous Provisions for unemployment insurance unemployment insurance offers more than just monthly benefits to help you find job for instance recipients can utilize up to 50% of their benefit to continue their education or training at an accredited Institute making them more marketable for future work this feature makes Germany's system truly unique amongst its counterparts across Europe four Germans are very punctual ah the Germans and their punctuality it's one of the foundations of life in Germany it doesn't matter if you're running late to a meeting eating dinner with friends or just taking the bus if you live in Germany and don't arrive on time be prepared to get some judging looks for those unfamiliar with German culture schedules and arrival times can seem like an implicit expectation people may start mentally counting down 5 minutes before your planned arrival time but being fashionably late is not accepted here the pros of living around punctual people first off no more waiting for late friends you'll see someone at 6:00 p.m. if they claim they'll be there the 5 minute delay phone calls are over five very livable if you're looking for a fantastic country to live Germany should be your first choice Germany has exceptional quality of life and some of the world's most beautiful cities and towns to begin life in Germany is safe Germany's low crime rate is due to its tight regulations and extensive security measures Germany's cultural offerings and safety make it livable from concerts and festivals like October Fest to wonderful galleries like alter naal Gallery Berlin you can explore fantastic new experiences and learn about European art and culture six efficient public transport Germany has very efficient public transport systems that's right no more worrying about finding a parking spot no more car payments and insurance costs instead you get to hop on a bus or train and take advantage of free Wi-Fi while you travel what's even better is that most cities offer discounts for students or those with limited incomes so everyone has access to Affordable transport options plus when it comes to safety there are cameras on all trains and buses throughout the country making sure that your journey is secure seven Germans are fairly honest and reliable living in Germany can be an absolute dream come true especially if you value honesty and reliability Germans have a well-deserved reputation for being fairly honest and reliable you get what you pay for this is one of the major pros of living here not only do people usually keep their word but they don't beat around the bush either when discussing something important or giving feedback that means that no matter who it comes from even your boss at work you can rest assured that what is said will not be sugarcoated it'll be straightforward and to the point cons one bureaucracy Germans think of beer gardens and sausages not paperwork unfortunately bureaucracy is an unavoidable aspect of life here from registering at the alander beura Foreigner's office to getting health insurance or a bank account expect a sea of forms remember to gather Department stamps and signatures along the Route navigating this bureaucratic maze can feel like every institution has its own rules making it harder for an outsider two difficulty in making friends one of the major cons is making friends yes it can be quite difficult to make friends in this culturally diverse country if you don't speak the same language or understand its Customs it is one of the major negative sides of living in Germany but don't be alarmed Germans do actually have their own way of connecting with others through beer gardens now before you go thinking that all German socializing centers around drinking beer let me clarify there are other activities too like playing cards and board games or just chatting away over dinner you may even find yourself jumping on trampolines at an outdoor party with your new found Buddies three bad customer service when living in Germany you're likely to come across some seriously bad customer service it can range from basic rudess all the way up to downright hostile behavior from staff at various establishments and companies here you might be wondering why this is well the Germans have their own unique culture when it comes to customer relations that doesn't always align with our expectations as customers outside Germany let me give you an example let's say you are out shopping and spot something that catches your eye so naturally you approach a shop assistant for help only to get hit with a very brusk response or even worse they just ignore you altogether this often happens because they expect customers to shop independently rather than actively seeking out assistance this is one of the most ignored negative sides of living in Germany four bad internet connection it's no surprised that moving to Germany has its advantages from their amazing culture and delicious food to the gorgeous view news but one thing that many people Overlook is how bad the internet connection can be here that's right folks you heard it here first living in Germany comes with a terrible internet connection it is one of the most annoying negative sides of living in Germany sure some areas of the country have better connections than others but on average Germans are stuck with slow speeds and high prices when it comes to their internet service providers or isps from dropped connections to painfully slow loading times let us tell you it's enough to make any techsavvy person throw their computer out the window five no air conditioning not everything is perfect here especially when it comes to air conditioning while other countries are cranking down their temperatures with powerful AC units Germany has been left a bit behind on this front which can make summer days quite uncomfortable no matter how hot it gets outside Germans just seem content with keeping their windows open and relying on fans or luck to stay cool indoors this means that if you're ever visiting or living in Germany during the summer months then you prepare yourself for some sweat filled days ahead luckily though there's usually a nice breeze coming through those open Windows sort of like Nature's Own built-in air conditioner six tough language if you thought learning a foreign language was hard think again if you've ever wanted to test your linguistic metal look no further than Germany from casual conversation to mastering the finer points of grammar and verb conjugation learning German can be quite the challenge this might be one of those negative sides of living in Germany that you can overcome as you live there and learn the language sure moving to Germany has its perks but one big con is that speaking the native tongue can sometimes feel like a Herculean task the sheer amount of vocabulary and complex sentence structure especially compared to English can make some expats feel intimidated or overwhelmed especially if they're starting from scratch or brushing up on their skills after years away from the classroom we have tried our best to discuss all the pros and cons of living in Germany in this video living in this diverse and Lively country offers great job prospects great infrastructure and a great quality of life however like any move to a new home in a new country there are downsides such as slow internet bureaucracy and missing your favorite comfort Foods before moving to Germany weigh all these perks and downsides thank you for watching this video please don't forget to comment subscribe and turn on the notification Bell
Germany has a huge problem its population is aging fast they're now around two workers for every pensioner the working population is shrinking as hundreds of thousands of baby boomers go into retirement Germany's federal employment agency says the country needs 400,000 skilled immigrants a year in Germany we need skilled Workforce we need additionally um skilled Workforce from abroad but whether immigrants f welcome is a whole other [Applause] [Music] question H so is Germany attractive enough f
"""

# Initialize ChromaDB
chroma_client = chromadb.Client()

# Create a collection in ChromaDB, if it doesn't already exist
collection = chroma_client.create_collection("youtube_video_transcripts3", get_or_create=True)

# Initialize the SentenceTransformer model for embedding text
sentence_model = SentenceTransformer('all-mpnet-base-v2')

def embed_text(text):
    """
    Convert text into embeddings using SentenceTransformer.
    """
    return sentence_model.encode([text])[0]

def upsert_to_chromadb(video_id, transcript_text):
    """
    Store video transcript embeddings in ChromaDB.
    """
    embedding = embed_text(transcript_text)
    collection.upsert(
        ids=[video_id],
        embeddings=[embedding],
        metadatas=[{"video_id": video_id, "transcript": transcript_text}]
    )

def get_video_transcript(video_id):
    """
    Fetch transcript for a given YouTube video ID.
    """
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = " ".join([item['text'] for item in transcript if 'text' in item])
        return transcript_text
    except Exception as e:
        print(f"Error retrieving transcript for video {video_id}: {e}")
        return None

def answer_question(question, video_id):
    """
    Answer a question based on the transcript of the selected video.
    """
    # Retrieve relevant video context using similarity search from ChromaDB
    question_embedding = embed_text(question)
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=1
    )

    # Print the results to understand its structure
    print("Results:", results)

    # Get the transcript text from the results
    context_text = results['metadatas'][0][0]['transcript']

    # Create a QA chain with the OpenAI model and the context
    qa_chain = LLMChain(llm=llm, prompt=qa_prompt)
    answer = qa_chain.run(context=combined_context, question=question)

    return answer

# Example usage with a list of video IDs
video_ids = ['2u4ItZerRac', 'I2zF1I60hPg', '8xqSF-uHCUs', 'LtmS-c1pChY', 'sJNxT-I7L6s']
transcripts = {}

# Fetch and store the transcript for each video
for video_id in video_ids:
    transcript = get_video_transcript(video_id)
    if transcript:
        transcripts[video_id] = transcript
        upsert_to_chromadb(video_id, transcript)
        print(f"Stored embedding for {video_id}")
    else:
        print(f"No transcript available for video {video_id}")

# Example query
question = "What is the main topic of the video?"
video_id = "2u4ItZerRac"
answer = answer_question(question, video_id)
print("Answer:", answer)

# Example query with refined prompt
question = "What are the benefits of immigration to Germany?"
answer = answer_question(question, video_id)
print("Answer:", answer)

question = "What are the challenges faced by immigrants in Germany?"
answer = answer_question(question, video_id)
print("Answer:", answer)


C:\Users\mehak\AppData\Local\Temp\ipykernel_17896\1416730173.py:36: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


Stored embedding for 2u4ItZerRac
Stored embedding for I2zF1I60hPg
Stored embedding for 8xqSF-uHCUs
Stored embedding for LtmS-c1pChY
Stored embedding for sJNxT-I7L6s
Results: {'ids': [['2u4ItZerRac']], 'embeddings': None, 'documents': [[None]], 'uris': None, 'data': None, 'metadatas': [[{'transcript': "[Music] the pros and cons of living in Germany today we're going to talk about the pros and cons of living in Germany Germany is an amazing country with a strong economy lots of culture great infrastructure and friendly people but that doesn't mean it's the right place for everyone in this video we'll look at all aspects of life in Germany so you can decide if it's the right move for you so let's get started Pros living in Germany is a great experience it offers many advantages both long and shortterm that make it an attractive destination for those considering making the move here are some of the pros of living in Germany one free education for more than 25 years Germany has offered tuit

C:\Users\mehak\AppData\Local\Temp\ipykernel_17896\1416730173.py:105: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm, prompt=qa_prompt)
C:\Users\mehak\AppData\Local\Temp\ipykernel_17896\1416730173.py:106: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(context=combined_context, question=question)


Answer: The main topic of the video is the pros and cons of living in Germany, discussing various aspects of life in the country to help viewers decide if moving to Germany is the right choice for them.
Results: {'ids': [['2u4ItZerRac']], 'embeddings': None, 'documents': [[None]], 'uris': None, 'data': None, 'metadatas': [[{'transcript': "[Music] the pros and cons of living in Germany today we're going to talk about the pros and cons of living in Germany Germany is an amazing country with a strong economy lots of culture great infrastructure and friendly people but that doesn't mean it's the right place for everyone in this video we'll look at all aspects of life in Germany so you can decide if it's the right move for you so let's get started Pros living in Germany is a great experience it offers many advantages both long and shortterm that make it an attractive destination for those considering making the move here are some of the pros of living in Germany one free education for more 

In [13]:
pip install yt-dlp

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install --upgrade whisper yt-dlp

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install --upgrade openai-whisper

Note: you may need to restart the kernel to use updated packages.


In [16]:
import subprocess
import platform

def install_libc():
    try:
        # Update package lists and install libc6 for Debian/Ubuntu
        if platform.system() == "Linux":
            distro = platform.linux_distribution()[0].lower()
            if "ubuntu" in distro or "debian" in distro:
                subprocess.run(["sudo", "apt-get", "update"], check=True)
                subprocess.run(["sudo", "apt-get", "install", "-y", "libc6"], check=True)
            elif "centos" in distro or "red hat" in distro or "fedora" in distro:
                subprocess.run(["sudo", "yum", "install", "-y", "glibc"], check=True)
            else:
                print("Unsupported Linux distribution.")
        else:
            print("This script only supports Linux distributions.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e}")

if __name__ == "__main__":
    install_libc()

This script only supports Linux distributions.


In [19]:
import os
import yt_dlp
import whisper
import subprocess

def download_audio(video_url, output_path):
    """
    Download audio from a YouTube video and save it as a .wav file.
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'ffmpeg_location': r'C:\Users\mehak\Downloads\ffmpeg-7.1-essentials_build\ffmpeg-7.1-essentials_build\bin'  # Ensure this path is correct
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])


def preprocess_audio(input_path, output_path):
    """
    Preprocess the audio file using ffmpeg to make it compatible with Whisper.
    Converts the audio to:
        - 16 kHz sample rate
        - Mono channel
        - 16-bit signed PCM
    """
    # Update this path to the exact location of ffmpeg.exe
    ffmpeg_path = r"C:\Users\mehak\Downloads\ffmpeg-7.1-essentials_build\bin\ffmpeg.exe"
    
    command = [
        ffmpeg_path,  # Use the full path to ffmpeg
        "-i", input_path,
        "-ar", "16000",  # Set sample rate to 16 kHz
        "-ac", "1",      # Convert to mono
        "-c:a", "pcm_s16le",  # Set codec to 16-bit signed PCM
        output_path
    ]
    try:
        subprocess.run(command, check=True)
        print(f"Audio preprocessed successfully: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error during audio preprocessing: {e}")


def transcribe_audio(audio_path):
    """
    Transcribe audio input into text using Whisper.
    """
    normalized_audio_path = os.path.normpath(audio_path)
    if not os.path.exists(normalized_audio_path):
        raise FileNotFoundError(f"Audio file not found: {normalized_audio_path}")
    
    print(f"Audio file found at: {normalized_audio_path}")
    
    whisper_model = whisper.load_model("base")
    result = whisper_model.transcribe(normalized_audio_path)
    return result['text']


# Paths
video_url = 'https://www.youtube.com/watch?v=dUU-vj4FnqU'
downloaded_audio_path = r'C:\Users\mehak\Desktop\AI-Engineering Bootcamp\multimodal-ai-chatbot-for-Youtubevideos--Immigrants-of-Germany-QA\output_audio.wav'
fixed_audio_path = r'C:\Users\mehak\Desktop\AI-Engineering Bootcamp\multimodal-ai-chatbot-for-Youtubevideos--Immigrants-of-Germany-QA\output_audio_fixed.wav'

# Step 1: Download audio from YouTube video
print("Downloading audio...")
download_audio(video_url, downloaded_audio_path)

# Verify download
if os.path.exists(downloaded_audio_path):
    print(f"File {downloaded_audio_path} downloaded successfully.")
else:
    raise FileNotFoundError(f"File {downloaded_audio_path} not found after download.")

# Step 2: Preprocess audio for Whisper
print("Preprocessing audio...")
preprocess_audio(downloaded_audio_path, fixed_audio_path)

# Verify preprocessing
if os.path.exists(fixed_audio_path):
    print(f"File {fixed_audio_path} preprocessed successfully.")
else:
    raise FileNotFoundError(f"File {fixed_audio_path} not found after preprocessing.")

# Step 3: Transcribe audio
print("Transcribing audio...")
try:
    transcribed_text = transcribe_audio(fixed_audio_path)
    print(f"Transcribed Text:\n{transcribed_text}")
except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=dUU-vj4FnqU
[youtube] dUU-vj4FnqU: Downloading webpage
[youtube] dUU-vj4FnqU: Downloading tv player API JSON
[youtube] dUU-vj4FnqU: Downloading ios player API JSON
[youtube] dUU-vj4FnqU: Downloading m3u8 information
[info] dUU-vj4FnqU: Downloading 1 format(s): 251
[download] C:\Users\mehak\Desktop\AI-Engineering Bootcamp\multimodal-ai-chatbot-for-Youtubevideos--Immigrants-of-Germany-QA\output_audio.wav has already been downloaded
[download] 100% of  135.94MiB
[ExtractAudio] Destination: C:\Users\mehak\Desktop\AI-Engineering Bootcamp\multimodal-ai-chatbot-for-Youtubevideos--Immigrants-of-Germany-QA\output_audio.wav
Deleting original file C:\Users\mehak\Desktop\AI-Engineering Bootcamp\multimodal-ai-chatbot-for-Youtubevideos--Immigrants-of-Germany-QA\output_audio.orig.wav (pass -k to keep)
File C:\Users\mehak\Desktop\AI-Engineering Bootcamp\multimodal-ai-chatbot-for-Youtubevideos--Immigrants-of-Germany-QA\output_audio.wav downloade

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [18]:
import gradio as gr

def chatbot_interface(question, video_title, audio_file):
    """
    Handle user queries and answer questions based on selected video.
    """
    video_choices = {
        'Video 1': '2u4ItZerRac',
        'Video 2': 'I2zF1I60hPg',
        'Video 3': '8xqSF-uHCUs',
        'Video 4': 'LtmS-c1pChY',
        'Video 5': 'sJNxT-I7L6s'
    }

    video_id = video_choices.get(video_title)

    if audio_file:
        # Transcribe audio if provided
        question = transcribe_audio(audio_file)

    if video_id:
        return answer_question(question, video_id)
    else:
        return "Please select a valid video."

# Create Gradio interface
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[
        gr.Textbox(label="Enter your question"),
        gr.Dropdown(label="Select Video", choices=['Video 1', 'Video 2', 'Video 3', 'Video 4']),
        gr.Audio(label="Upload Audio (optional)", type="file", optional=True)
    ],
    outputs="text"
)

# Launch the Gradio app
iface.launch(share=True)

TypeError: Audio.__init__() got an unexpected keyword argument 'optional'

In [16]:
pip install transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [17]:
# evaluation and fine tuning
# Define a set of test queries and expected answers
test_cases = [
    {
        "question": "What are the benefits of immigration to Germany?",
        "expected_answer": "The benefits of immigration to Germany include: 1. **Free Education**: Germany offers tuition-free higher education at public universities, allowing students from around the world to pursue their degrees without the burden of expensive tuition costs. There are also grants and scholarships available for certain fields that can cover living expenses. 2. **High Quality of Life**: Germany is known for its exceptional quality of life, with beautiful cities, a low crime rate, and a safe environment. The country provides a rich cultural experience with various festivals, concerts, and art galleries. **Efficient Public Transport**: Germany has a highly efficient public transport system, making it easy to travel without the need for a car. This includes affordable options for students and safety measures like surveillance on public transport. 4. **Generous Unemployment Benefits**: The German government provides substantial unemployment insurance, which not only offers financial support but also allows recipients to use part of their benefits for further education or training. 5. **Punctuality and Reliability**: The culture in Germany values punctuality and reliability, which can lead to a more organized and predictable lifestyle. 6. **Drinkable Tap Water**: The tap water in Germany is generally safe and of high quality, often considered better than in many other countries. 7. **Cultural Diversity**: Immigrants can experience a culturally diverse environment, which can enrich their lives and provide opportunities for personal growth and new friendships.ese benefits make Germany an attractive destination for immigrants seeking new opportunities and a high standard of living."
    },
    {
        "question": "What are the challenges faced by immigrants in Germany?",
        "expected_answer": "Immigrants in Germany face several challenges, including:1 **Bureaucracy**: Navigating the complex bureaucratic processes can be overwhelming, as it involves dealing with numerous forms, registrations, and obtaining various approvals, which can be particularly daunting for newcomers. 2. **Difficulty in Making Friends**: Building social connections can be challenging, especially for those who do not speak German or are unfamiliar with the local customs. This can lead to feelings of isolation. 3. **Language Barrier**: Learning German can be a significant hurdle for immigrants, as the language has complex grammar and vocabulary. This can make everyday interactions and integration into society more difficult. 4. **Cultural Differences**: Understanding and adapting to German social norms and customs can be challenging, which may hinder social interactions and integration. 5. **Perception of Unwelcomeness**: In some areas, particularly where anti-immigration sentiments are strong, immigrants may feel unwelcome or face discrimination, which can affect their sense of safety and belonging. 6. **Bad Customer Service**: Immigrants may encounter poor customer service, as the cultural expectations around customer relations in Germany can differ significantly from those in other countries, leading to misunderstandings and frustration.These challenges can impact the overall experience of immigrants in Germany, making it essential for them to prepare and seek support as they navigate their new environment."
    },
    # Add more test cases as needed
]

def evaluate_model(test_cases):
    """
    Evaluate the model with a set of test cases.
    """
    correct_answers = 0
    for test in test_cases:
        answer = answer_question(test['question'], "2u4ItZerRac, I2zF1I60hPg, 8xqSF-uHCUs, LtmS-c1pCh")  # Assuming using one video for testing
        print(f"Question: {test['question']}")
        print(f"Expected: {test['expected_answer']}")
        print(f"Received: {answer}")
        if test['expected_answer'].lower() in answer.lower():
            correct_answers += 1
        print()
    
    accuracy = correct_answers / len(test_cases)
    print(f"Model accuracy: {accuracy * 100}%")

# Run the evaluation
evaluate_model(test_cases)

Results: {'ids': [['2u4ItZerRac']], 'embeddings': None, 'documents': [[None]], 'uris': None, 'data': None, 'metadatas': [[{'transcript': "[Music] the pros and cons of living in Germany today we're going to talk about the pros and cons of living in Germany Germany is an amazing country with a strong economy lots of culture great infrastructure and friendly people but that doesn't mean it's the right place for everyone in this video we'll look at all aspects of life in Germany so you can decide if it's the right move for you so let's get started Pros living in Germany is a great experience it offers many advantages both long and shortterm that make it an attractive destination for those considering making the move here are some of the pros of living in Germany one free education for more than 25 years Germany has offered tuition-free higher education at public universities with no charge for most courses and little no cost for administrative fees this means that students coming from all 

Evaluation and fine tuning 

In [18]:
combined_context = """
Transcript for 2u4ItZerRac:
[Music] The pros and cons of living in Germany. Today we're going to talk about the pros and cons of living in Germany. Germany is an amazing country with a strong economy, lots of culture, great infrastructure, and friendly people, but that doesn't mean it's the right place for everyone. In this video, we'll look at all aspects of life in Germany so you can decide if it's the right move for you. So let's get started. Pros: living in Germany is a great experience. It offers many advantages both long and short-term that make it an attractive destination for those considering making the move. Here are some of the pros of living in Germany: 1. Free education: For more than 25 years, Germany has offered tuition-free higher education at public universities, with no charge for most courses and little to no cost for administrative fees. This means that students from all over the world can pursue their degree programs without worrying about expensive tuition costs. Moreover, those studying in certain fields even have access to grants and scholarships which cover living expenses as well. In addition to this incredible benefit, German universities also provide high-quality instruction from some of the finest academic minds in Europe. With famous institutions such as Humboldt University Berlin leading the way in research and innovation, you are sure to receive an excellent standard of learning whether you're taking up a course on engineering or literature. 2. Drinkable tap water: Living in Germany certainly comes with a lot of perks, from the delicious sausages to the breathtaking cities, but one perk that doesn't often get mentioned is how drinkable tap water in Germany is. It's true, and it's really quite remarkable. Of course, not all German towns have perfect tap water, but by far most do. In fact, many Germans believe their drinking water is so good because it's monitored more stringently than in other countries. And for sure, there are no worries about added chemicals or nasty tastes. What could be better? 3. Unemployment benefits: If you find yourself out of work due to whatever reason, don't fret. Your well-being is taken care of thanks to the German government's generous provisions for unemployment insurance. Unemployment insurance offers more than just monthly benefits to help you find a job. For instance, recipients can utilize up to 50% of their benefit to continue their education or training at an accredited institute, making them more marketable for future work. This feature makes Germany's system truly unique amongst its counterparts across Europe. 4. Punctuality: Germans are very punctual. It's one of the foundations of life in Germany. It doesn't matter if you're running late to a meeting, eating dinner with friends, or just taking the bus. If you live in Germany and don't arrive on time, be prepared to get some judging looks. For those unfamiliar with German culture, schedules and arrival times can seem like an implicit expectation. People may start mentally counting down 5 minutes before your planned arrival time, but being fashionably late is not accepted here. The pros of living around punctual people: first off, no more waiting for late friends. You'll see someone at 6:00 p.m. if they claim they'll be there. The 5-minute delay phone calls are over. 5. Very livable: If you're looking for a fantastic country to live in, Germany should be your first choice. Germany has exceptional quality of life and some of the world's most beautiful cities and towns to begin with. Life in Germany is safe. Germany's low crime rate is due to its tight regulations and extensive security measures. Germany's cultural offerings and safety make it livable. From concerts and festivals like Oktoberfest to wonderful galleries like the Alte Nationalgalerie Berlin, you can explore fantastic new experiences and learn about European art and culture. 6. Efficient public transport: Germany has very efficient public transport systems. That's right, no more worrying about finding a parking spot, no more car payments and insurance costs. Instead, you get to hop on a bus or train and take advantage of free Wi-Fi while you travel. What's even better is that most cities offer discounts for students or those with limited incomes, so everyone has access to affordable transport options. Plus, when it comes to safety, there are cameras on all trains and buses throughout the country, making sure that your journey is secure. 7. Honest and reliable people: Germans are fairly honest and reliable. Living in Germany can be an absolute dream come true, especially if you value honesty and reliability. Germans have a well-deserved reputation for being fairly honest and reliable. You get what you pay for. This is one of the major pros of living here. Not only do people usually keep their word, but they don't beat around the bush either when discussing something important or giving feedback. That means that no matter who it comes from, even your boss at work, you can rest assured that what is said will not be sugarcoated. It'll be straightforward and to the point. Cons: 1. Bureaucracy: Germans think of beer gardens and sausages, not paperwork. Unfortunately, bureaucracy is an unavoidable aspect of life here. From registering at the Ausländerbehörde (foreigner's office) to getting health insurance or a bank account, expect a sea of forms. Remember to gather department stamps and signatures along the route. Navigating this bureaucratic maze can feel like every institution has its own rules, making it harder for an outsider. 2. Difficulty in making friends: One of the major cons is making friends. Yes, it can be quite difficult to make friends in this culturally diverse country if you don't speak the same language or understand its customs. It is one of the major negative sides of living in Germany, but don't be alarmed. Germans do actually have their own way of connecting with others through beer gardens. Now, before you go thinking that all German socializing centers around drinking beer, let me clarify. There are other activities too, like playing cards and board games or just chatting away over dinner. You may even find yourself jumping on trampolines at an outdoor party with your newfound buddies. 3. Bad customer service: When living in Germany, you're likely to come across some seriously bad customer service. It can range from basic rudeness all the way up to downright hostile behavior from staff at various establishments and companies here. You might be wondering why this is. Well, the Germans have their own unique culture when it comes to customer relations that doesn't always align with our expectations as customers outside Germany. Let me give you an example. Let's say you are out shopping and spot something that catches your eye. So naturally, you approach a shop assistant for help, only to get hit with a very brusque response, or even worse, they just ignore you altogether. This often happens because they expect customers to shop independently rather than actively seeking out assistance. This is one of the most ignored negative sides of living in Germany. 4. Bad internet connection: It's no surprise that moving to Germany has its advantages, from their amazing culture and delicious food to the gorgeous views, but one thing that many people overlook is how bad the internet connection can be here. That's right, folks, you heard it here first. Living in Germany comes with a terrible internet connection. It is one of the most annoying negative sides of living in Germany. Sure, some areas of the country have better connections than others, but on average, Germans are stuck with slow speeds and high prices when it comes to their internet service providers or ISPs. From dropped connections to painfully slow loading times, let us tell you, it's enough to make any tech-savvy person throw their computer out the window. 5. No air conditioning: Not everything is perfect here, especially when it comes to air conditioning. While other countries are cranking down their temperatures with powerful AC units, Germany has been left a bit behind on this front, which can make summer days quite uncomfortable. No matter how hot it gets outside, Germans just seem content with keeping their windows open and relying on fans or luck to stay cool indoors. This means that if you're ever visiting or living in Germany during the summer months, then you should prepare yourself for some sweat-filled days ahead. Luckily though, there's usually a nice breeze coming through those open windows, sort of like nature's own built-in air conditioner. 6. Tough language: If you thought learning a foreign language was hard, think again. If you've ever wanted to test your linguistic mettle, look no further than Germany. From casual conversation to mastering the finer points of grammar and verb conjugation, learning German can be quite the challenge. This might be one of those negative sides of living in Germany that you can overcome as you live there and learn the language. Sure, moving to Germany has its perks, but one big con is that speaking the native tongue can sometimes feel like a Herculean task. The sheer amount of vocabulary and complex sentence structure, especially compared to English, can make some expats feel intimidated or overwhelmed, especially if they're starting from scratch or brushing up on their skills after years away from the classroom. We have tried our best to discuss all the pros and cons of living in Germany in this video. Living in this diverse and lively country offers great job prospects, great infrastructure, and a great quality of life. However, like any move to a new home in a new country, there are downsides such as slow internet, bureaucracy, and missing your favorite comfort foods. Before moving to Germany, weigh all these perks and downsides. Thank you for watching this video. Please don't forget to comment, subscribe, and turn on the notification bell.

Transcript for I2zF1I60hPg:
should you move to Germany that's an easy question to a hard answer but I'm going to try my [Music] best hi my name is Fernando I have been living in Munich Germany for eight years now and yeah I have a lot of experience I say and I'm very happy to kind of like give you some ideas and some stories to will probably help you make a better decision if you should move to Germany or not the first thing of course is that there are many reasons why you are probably moving to Germany either you got a a 

Transcript for 8xqSF-uHCUs:
here in Germany the government is meeting this week to discuss new measures to more easily Deport migrants whose Asylum application was rejected or who have been convicted of certain crimes it's a controversial Topic in some areas of Germany but not in parts of the East where the anti-immigration alternative for Germany is the strongest political party many newly arrived migrants there worry that they are no longer welcome here because of war and I want to have a safe life here I'm 100% confiden

Transcript for LtmS-c1pChY:
the alternative for Germany afd party has unveiled a radical new approach to immigration policy signaling a potential shift in the nation's migration landscape Party leaders have proposed stringent measures that could dramatically reshape Germany's approach to foreign Nationals and border control afd leadership has outlined a comprehensive strategy targeting what they describe as excessive immigration the proposed plan includes significant restrictions on Asylum Seekers family reunification and
"""

In [19]:
questions_answers = [
    {
        "question": "What are the benefits of immigration to Germany?",
        "expected_answer": "Immigration brings cultural diversity, fills labor shortages, and contributes to economic growth."
    },
    {
        "question": "What are the challenges faced by immigrants in Germany?",
        "expected_answer": "Immigrants may face language barriers, integration difficulties, and potential discrimination."
    }
]

In [20]:
from datasets import Dataset

data = {
    "context": [combined_context] * len(questions_answers),
    "question": [qa["question"] for qa in questions_answers],
    "answer": [qa["expected_answer"] for qa in questions_answers]
}

dataset = Dataset.from_dict(data)

In [21]:
pip install transformers datasets

In [22]:
pip install transformers [torch]

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '[torch]': Expected package name at the start of dependency specifier
    [torch]
    ^


In [23]:
!pip install transformers datasets accelerate

In [ ]:
# Import necessary libraries
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorWithPadding
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Get the Hugging Face API token from the environment variable
hf_api_token = os.getenv("HUGGINGFACE_API_KEY")

# Ensure the API token is loaded
if hf_api_token is None:
    raise ValueError("Hugging Face API token not found in environment variables.")

# Combined context from the provided transcripts
combined_context = """
Transcript for 2u4ItZerRac:
[Music] The pros and cons of living in Germany. Today we're going to talk about the pros and cons of living in Germany...

# Add the other transcripts here
"""

# Define questions and expected answers
questions_answers = [
    {
        "question": "What are the benefits of immigration to Germany?",
        "expected_answer": "Immigration brings cultural diversity, fills labor shortages, and contributes to economic growth."
    },
    {
        "question": "What are the challenges faced by immigrants in Germany?",
        "expected_answer": "Immigrants may face language barriers, integration difficulties, and potential discrimination."
    }
]

# Create a dataset
data = {
    "context": [combined_context] * len(questions_answers),
    "question": [qa["question"] for qa in questions_answers],
    "answer": [qa["expected_answer"] for qa in questions_answers]
}
dataset = Dataset.from_dict(data)

# Load pre-trained model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_api_token)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_api_token)

# Add padding token to the tokenizer if it doesn't have one
if tokenizer.pad_token is 50257:
    tokenizer.add_special_tokens({'pad_token': '[50257]'})
    model.resize_token_embeddings(len(tokenizer))

# Tokenize dataset
def tokenize_function(examples):
    inputs = [f"Context: {context}\n\nQuestion: {question}\n\nAnswer:" for context, question in zip(examples["context"], examples["question"])]
    targets = [answer for answer in examples["answer"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

<>:51: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:51: SyntaxWarning: "is" with a literal. Did you mean "=="?
c:\Users\mehak\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\auto\tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\mehak\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\auto\auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\mehak\AppData\Local\Temp\ipykernel_12016\1179161985.py:51: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if tokenizer.pad_token is 50257:
Map:   0%|          | 0/2 [00:00<?, ? examples/s]


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

Step 5: Evaluate the Model
Evaluate the fine-tuned model using the same questions and calculate the metrics:



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the fine-tuned model and tokenizer
fine_tuned_model = AutoModelForCausalLM.from_pretrained("fine_tuned_model")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model")

# Generate predictions using the fine-tuned model
def generate_answer(question, context):
    input_text = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    inputs = fine_tuned_tokenizer.encode(input_text, return_tensors="pt")
    outputs = fine_tuned_model.generate(inputs, max_length=512, num_return_sequences=1)
    answer = fine_tuned_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example evaluation
predicted_answers = [generate_answer(qa["question"], combined_context) for qa in questions_answers]
true_answers = [qa["expected_answer"] for qa in questions_answers]

# Calculate metrics
accuracy = accuracy_score(true_answers, predicted_answers)
precision = precision_score(true_answers, predicted_answers, average='weighted')
recall = recall_score(true_answers, predicted_answers, average='weighted')
f1 = f1_score(true_answers, predicted_answers, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")